# Creating a basic chat experience with kernel arguments

In this chat scenario, as the user talks back and forth with the bot, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. 

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.1.0"
#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.1.0

Let's define the prompt.

In [2]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'Beats me, I'm just here for the drinks!' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

Register your semantic function

In [3]:
var chatFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings);

Initialize your arguments

In [4]:
var history = "";
var arguments = new KernelArguments()
{
    ["history"] = history
};

Chat with the Bot

In [5]:
var userInput = "Hi, I'm looking for cocktail suggestions";
arguments["userInput"] = userInput;

var bot_answer = await chatFunction.InvokeAsync(kernel, arguments);

Update the history with the output and set this as the new input value for the next request

In [6]:
history += $"\nUser: {userInput}\nAI: {bot_answer}\n";
arguments["history"] = history;

Console.WriteLine(history);


User: Hi, I'm looking for cocktail suggestions
AI: Sure! What type of cocktail are you in the mood for?



Keep Chatting!

In [7]:
Func<string, Task> Chat = async (string input) => {
    // Save new message in the arguments
    arguments["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(kernel, arguments);

    // Append the new interaction to the chat history
    var result = $"\nUser: {input}\nAI: {answer}\n";
    history += result;

    arguments["history"] = history;
    
    // Show the response
    Console.WriteLine(result);
};

In [8]:
await Chat("I would like a cocktail suggestion for breakfast. Please list one.");


User: I would like a cocktail suggestion for breakfast. Please list one.
AI: How about a classic Mimosa? It's made with champagne and orange juice, and it's perfect for a brunch or breakfast cocktail.



In [ ]:
await Chat("that sounds interesting, how do you make one?");

In [9]:
await Chat("Which topic from the ones you listed do you think most people find interesting?");


User: Which topic from the ones you listed do you think most people find interesting?
AI: Beats me, I'm just here for the drinks!



In [ ]:
await Chat("could you list some more cocktails I could try about the topic(s) mentioned?");

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [10]:
Console.WriteLine(history);


User: Hi, I'm looking for cocktail suggestions
AI: Sure! What type of cocktail are you in the mood for?

User: I would like a cocktail suggestion for breakfast. Please list one.
AI: How about a classic Mimosa? It's made with champagne and orange juice, and it's perfect for a brunch or breakfast cocktail.

User: Which topic from the ones you listed do you think most people find interesting?
AI: Beats me, I'm just here for the drinks!

